One of our ideas was to use google's pretrained model and create our specific model with its help.

In [1]:
!pip install keras_tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 3.4 MB/s eta 0:00:00


In [2]:
import numpy as np
import tensorflow as tf
import os
import librosa
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch
from keras_tuner.tuners import Hyperband
from keras import models

Getting the training files and unzipping them like in the other notebook

In [3]:
!wget -O training_files.zip https://www.dropbox.com/scl/fi/2yogrqyao6uypiwtbv8mc/file.zip?rlkey=e3bdpmjvfios021zxlo1ru3b4&dl=0

--2023-11-28 22:02:09--  https://www.dropbox.com/scl/fi/2yogrqyao6uypiwtbv8mc/file.zip?rlkey=e3bdpmjvfios021zxlo1ru3b4
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5acc3426b34b2b8978b42e81cb.dl.dropboxusercontent.com/cd/0/inline/CIZ0L_1Kby95h8JFBavNteYA1CEV66GJ4IPkQmZwuqgYlFruA6UZjQWMf-WeYJvIwomM2pKAtFdXxV6xa62Zx8yeJNflfDDfl2zTN08uYAro8ZhKaiIj69RzVj1P1HL2p7Obc_TVJgQndKrThasAzDA7/file# [following]
--2023-11-28 22:02:10--  https://uc5acc3426b34b2b8978b42e81cb.dl.dropboxusercontent.com/cd/0/inline/CIZ0L_1Kby95h8JFBavNteYA1CEV66GJ4IPkQmZwuqgYlFruA6UZjQWMf-WeYJvIwomM2pKAtFdXxV6xa62Zx8yeJNflfDDfl2zTN08uYAro8ZhKaiIj69RzVj1P1HL2p7Obc_TVJgQndKrThasAzDA7/file
Resolving uc5acc3426b34b2b8978b42e81cb.dl.dropboxusercontent.com (uc5acc3426b34b2b8978b42e81cb.dl.dropboxusercontent.com)... 162.125.3.15, 2620:100:6

In [4]:
!unzip training_files.zip

A streamkimeneten csak az utolsó 5000 sor látható.
  inflating: wavfiles/whbtit5/whbtit5_110495.wav  
  inflating: wavfiles/whbtit5/whbtit5_110472.wav  
   creating: wavfiles/moccha1/
  inflating: wavfiles/moccha1/moccha1_75146.wav  
  inflating: wavfiles/moccha1/moccha1_75065.wav  
  inflating: wavfiles/moccha1/moccha1_74888.wav  
  inflating: wavfiles/moccha1/moccha1_75042.wav  
  inflating: wavfiles/moccha1/moccha1_75399.wav  
  inflating: wavfiles/moccha1/moccha1_75360.wav  
  inflating: wavfiles/moccha1/moccha1_75192.wav  
  inflating: wavfiles/moccha1/moccha1_75303.wav  
  inflating: wavfiles/moccha1/moccha1_74947.wav  
  inflating: wavfiles/moccha1/moccha1_75176.wav  
  inflating: wavfiles/moccha1/moccha1_75278.wav  
  inflating: wavfiles/moccha1/moccha1_74897.wav  
  inflating: wavfiles/moccha1/moccha1_74877.wav  
  inflating: wavfiles/moccha1/moccha1_74989.wav  
  inflating: wavfiles/moccha1/moccha1_74925.wav  
  inflating: wavfiles/moccha1/moccha1_74988.wav  
  inflating: wav

The file path reader function is also the same as in the other notebook.

In [5]:
def read_file_paths(main_dictionary):
    main_directory = main_dictionary
    file_paths = []

    # go through all folders and get the paths of all .ogg audio files
    for root, directories, files in os.walk(main_directory):
        for file in files:
            if file.endswith('.ogg') or file.endswith('.wav'):
                file_path = os.path.join(root, file)
                file_paths.append(file_path)

    # os.walk may not go in alphabetical order thus it needs to be sorted
    file_paths.sort()
    return file_paths

In [6]:
filepaths = read_file_paths('wavfiles')

After we got the file paths, we need to extract the waveforms and labels from them.

In [18]:
def extract_label(filename):
    return filename.split('_')[0]

def load_and_convert(file_path, target_sr=32000):
    y, sr = librosa.load(file_path, sr=target_sr)
    return y, sr

def process_batch(batch_files):
    features = []
    labels = []
    for file in batch_files:
        label = extract_label(file)
        waveform, sr = load_and_convert(file)
        waveform = waveform.astype(np.float16)
        if(len(waveform) == 160000):
          features.append(waveform)
          labels.append(label)
    features = np.array(features)
    labels = np.array(labels)


    return features, labels


file_list = filepaths
features, labels = process_batch(filepaths)

In [21]:
print(features.shape)
print(labels.shape)

(5357, 160000)
(5357,)


The model will be loaded with using tensorflow_hub

In [9]:
import tensorflow_hub as hub

Now we load the pretrained model we used for this training variation.

In [10]:
model = hub.load('https://www.kaggle.com/models/google/bird-vocalization-classifier/frameworks/TensorFlow2/variations/bird-vocalization-classifier/versions/4')

The idea is to use the embedding vector that the model returns for every input to train our model. To do this, at first we iterate through the waveforms and create all the embeddings vectors.
The model only accepts (,160000) shaped inputs, that's why we needed to filter for 160000 length waveforms (5x32000 = 160000 and these are 5s long 32 Khz files)

In [11]:
X = []
i=0
for data in features:
  logits, embeddings = model.infer_tf(data[np.newaxis, :])
  print(f"processed {i} / {len(features)}")
  i+=1
  X.append(embeddings)

A streamkimeneten csak az utolsó 5000 sor látható.
processed 357 / 5357
processed 358 / 5357
processed 359 / 5357
processed 360 / 5357
processed 361 / 5357
processed 362 / 5357
processed 363 / 5357
processed 364 / 5357
processed 365 / 5357
processed 366 / 5357
processed 367 / 5357
processed 368 / 5357
processed 369 / 5357
processed 370 / 5357
processed 371 / 5357
processed 372 / 5357
processed 373 / 5357
processed 374 / 5357
processed 375 / 5357
processed 376 / 5357
processed 377 / 5357
processed 378 / 5357
processed 379 / 5357
processed 380 / 5357
processed 381 / 5357
processed 382 / 5357
processed 383 / 5357
processed 384 / 5357
processed 385 / 5357
processed 386 / 5357
processed 387 / 5357
processed 388 / 5357
processed 389 / 5357
processed 390 / 5357
processed 391 / 5357
processed 392 / 5357
processed 393 / 5357
processed 394 / 5357
processed 395 / 5357
processed 396 / 5357
processed 397 / 5357
processed 398 / 5357
processed 399 / 5357
processed 400 / 5357
processed 401 / 5357
proc

We need to reshape the array

In [12]:
X = np.array(X)
X_reshaped = X.reshape((X.shape[0], 1280))
print(X_reshaped.shape)

(5357, 1280)


In [13]:
!pip install keras_tuner

In [15]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras_tuner import Hyperband

In [22]:
label_encoder = LabelEncoder()
y_train_numerical = label_encoder.fit_transform(labels)


onehot_encoder = OneHotEncoder(sparse=False)
y_train_one_hot = onehot_encoder.fit_transform(y_train_numerical.reshape(-1, 1))



# Split the data into training and temporary sets (80% training, 20% temporary)
X_train, X_test, y_train, y_test = train_test_split(
    X_reshaped, y_train_one_hot, test_size=0.2, random_state=42
)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
def build_model(hp):
    model = models.Sequential()

    # Choose the number of hidden layers and units
    hp_units = hp.Int('units', min_value=32, max_value=1024, step=128)
    model.add(layers.Dense(units=hp_units, activation='relu', input_shape=(1280,)))

    # Add a variable number of hidden layers
    for _ in range(hp.Int('num_layers', 1, 20)):
        model.add(layers.Dense(units=hp_units, activation='relu'))

    # Add the output layer with softmax activation for classification
    num_classes = 262  # Adjust based on the number of classes in your classification task
    model.add(layers.Dense(num_classes, activation='softmax'))

    # Compile the model with an optimizer, learning rate, and loss function
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Define the tuner
tuner = Hyperband(
    build_model,
    objective='val_accuracy',
    factor=3,
    max_epochs=30,
    directory='output',
    project_name='BirdVoiceClass')

# Assuming you have a dataset with corresponding labels
# X_train, y_train = ...  # Your training data
# X_val, y_val = ...      # Your validation data
tuner.search(X_train, y_train, epochs=10, validation_split=0.1)

# Get the best model and hyperparameters
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

print("Best Hyperparameters:")
print(best_hyperparameters.get_config()['values'])


Trial 65 Complete [00h 00m 23s]
val_accuracy: 0.0419580414891243

Best val_accuracy So Far: 0.8228438496589661
Total elapsed time: 00h 19m 51s

Search: Running Trial #66

Value             |Best Value So Far |Hyperparameter
160               |288               |units
17                |1                 |num_layers
0.0001            |0.001             |learning_rate
4                 |10                |tuner/epochs
0                 |4                 |tuner/initial_epoch
2                 |3                 |tuner/bracket
0                 |2                 |tuner/round

Epoch 1/4


Here colab died again but as it is displayed after 66 trials the best validation accuracy was 0.82, which isnt bad